In [1]:
import os
import pickle
from glob import iglob
import numpy as np
import librosa

In [2]:
DATA_AUDIO_DIR = './heartbeat_data'
TARGET_SR = 8000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')
AUDIO_LENGTH = 10000

In [3]:
class_ids = {
    'normal': 0,
    'mumur': 1,
    'extrahls': 2,
    'artifact': 3,
    'unlabelled': 4,
}

In [5]:
def extract_class_id(wav_filename):
    if 'normal' in wav_filename:
        return class_ids.get('normal')
    elif 'murmur' in wav_filename:
        return class_ids.get('murmur')
    elif 'extrahls' in wav_filename:
        return class_ids.get('extrahls')
    elif 'artifact' in wav_filename:
        return class_ids.get('artifact')
    elif 'unlabelled' in wav_filename:
        return class_ids.get('unlabelled')
    else:
        return class_ids.get('unlabelled')

In [13]:
def read_audio_from_filename(filename, target_sr):
    audio,_ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

In [16]:
def convert_data():
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_DIR, '**/**.wav'), recursive=True)):
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # nomralize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original))))
            print('PAD New Length = ', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            print('CUT New length = ', len(audio_buf))
            
        output_folder = OUTPUT_DIR_TRAIN
        if i // 50 == 0:
            output_folder = OUTPUT_DIR_TEST
        
        output_filename = os.path.join(output_folder, str(i) + ".pkl")
        out = {
            'class_id': class_id,
            'audio': audio_buf,
            'sr': TARGET_SR
        }
        print(out)
        with open(output_filename, 'wb') as w:
            pickle.dump(out, w)

In [17]:
convert_data()

0 ./heartbeat_data/set_a/murmur__201101180902.wav 64276 0.0 1.0
CUT New length =  10000
{'class_id': None, 'audio': array([[-0.3388304 ],
       [-0.6199059 ],
       [-0.5296698 ],
       ...,
       [-0.4140227 ],
       [-0.40245813],
       [-0.39387414]], dtype=float32), 'sr': 8000}


FileNotFoundError: [Errno 2] No such file or directory: './output/test/0.pkl'